In [135]:
import os
import sys
import pandas as pd
import numpy as np
import re

from tqdm import tqdm
from pandas import Series, DataFrame

## 声明：itemid_to_variable_map.csv既包含mimic iii 的d_items表的内容， 又包含d_labitems表的内容。
比如itemid_to_variable_map.csv LEVEL2为"Albumin"的第三条数据的itemid是50862，来自于d_labitems表；

第四条数据的itemid是772，来自于d_items表。

## itemid_to_variable_map.csv的MIMIC LABEL列是两个表的LABEL的值
## d_labitems表的组成是：

ROW_ID	ITEMID	LABEL	FLUID	CATEGORY	LOINC_CODE	


## d_items表的组成是：
ROW_ID	ITEMID	LABEL	ABBREVIATION	DBSOURCE	LINKSTO	CATEGORY	UNITNAME	PARAM_TYPE	CONCEPTID	

其中，LINKSTO值为chartevents


In [136]:
subjects_root_path = 'data/root'
variable_map_file = '../resources/itemid_to_variable_map.csv'  # CSV containing ITEMID-to-VARIABLE map.
reference_range_file = '../resources/variable_ranges.csv'      # CSV containing reference ranges for VARIABLEs

In [137]:
variable_range = pd.read_csv(reference_range_file)
print(variable_range[variable_range['LEVEL2'] =='Height' ])

itemid_map_variable = pd.read_csv(variable_map_file)
print(itemid_map_variable.columns)
print(itemid_map_variable.COUNT)
print(itemid_map_variable)

    LEVEL2 LEVEL1  OUTLIER LOW  VALID LOW  IMPUTE  VALID HIGH  OUTLIER HIGH
27  Height    NaN          0.0        0.0   170.0       240.0         275.0
Index(['LEVEL2', 'LEVEL1', 'ALTERNATIVE', 'STATUS', 'STATUS NOTE', 'ITEMID',
       'MIMIC LABEL', 'UNITNAME', 'LINKSTO', 'COUNT', 'CATEGORY', 'CONCEPTID',
       'FLUID', 'LOINC_CODE', 'DBSOURCE', 'Unnamed: 15', 'PARAM_TYPE', 'NOTE'],
      dtype='object')
0        219475
1         41594
2         37625
3        146697
4         31022
          ...  
13228         0
13229         0
13230         0
13231         0
13232         0
Name: COUNT, Length: 13233, dtype: int64
                         LEVEL2                    LEVEL1  \
0      Alanine aminotransferase  Alanine aminotransferase   
1      Alanine aminotransferase  Alanine aminotransferase   
2      Alanine aminotransferase  Alanine aminotransferase   
3                       Albumin                   Albumin   
4                       Albumin                   Albumin   
...    

 ## 去掉itemid_to_variable_map.csv里面LEVEL2为空、或者COUNT<=0的数据，这样就得到了17个变量的name和itemid
### paper 里面说：the first column （LEVEL2）is the name of the variable in our benchmark
 ### COUNT 是什么？LEVEL1又是什么？

In [138]:
def read_itemid_to_variable_map(fn, variable_column='LEVEL2'):
    var_map = pd.read_csv(fn, index_col=None).fillna('').astype(str)
    # var_map[variable_column] = var_map[variable_column].apply(lambda s: s.lower())
    LAEVL_2_NULL_NUM_MASK = var_map[variable_column] == ''
    #print(f'var_map中 LEVEL2 值为null的数据有 {len(var_map[LAEVL_2_NULL_NUM_MASK])}个 ')
    var_map.COUNT = var_map.COUNT.astype(int)
    # ???COUNT是什么意思
    var_map = var_map[(var_map[variable_column] != '') & (var_map.COUNT > 0)]
    # ready代表什么？怎么来的？
    var_map = var_map[(var_map.STATUS == 'ready')]
    var_map.ITEMID = var_map.ITEMID.astype(int)
    var_map = var_map[[variable_column, 'ITEMID', 'MIMIC LABEL']].set_index('ITEMID')
    return var_map.rename({variable_column: 'VARIABLE', 'MIMIC LABEL': 'MIMIC_LABEL'}, axis=1)


In [139]:
itemid_map_variable  = read_itemid_to_variable_map(variable_map_file, variable_column='LEVEL2')
itemid_map_variable

,VARIABLE,MIMIC_LABEL
ITEMID,,
3348,Capillary refill rate,Capillary Refill
115,Capillary refill rate,Capillary Refill [Right]
8377,Capillary refill rate,Capillary Refill [Left]
8368,Diastolic blood pressure,Arterial BP [Diastolic]
220051,Diastolic blood pressure,Arterial Blood Pressure diastolic
...,...,...
3580,Weight,Present Weight (kg)
3693,Weight,Weight Kg
3581,Weight,Present Weight (lb)


## 得到17个所使用的临床变量

In [140]:
# 得到17个所使用的临床变量
variables = itemid_map_variable.VARIABLE.unique()
variables

array(['Capillary refill rate', 'Diastolic blood pressure',
       'Fraction inspired oxygen', 'Glascow coma scale eye opening',
       'Glascow coma scale motor response', 'Glascow coma scale total',
       'Glascow coma scale verbal response', 'Glucose', 'Heart Rate',
       'Height', 'Mean blood pressure', 'Oxygen saturation', 'pH',
       'Respiratory rate', 'Systolic blood pressure', 'Temperature',
       'Weight'], dtype=object)

In [141]:
#os.listdir(subjects_root_path)
len(os.listdir(subjects_root_path))

33798

In [142]:
def read_stays(subject_path):
    stays = pd.read_csv(os.path.join(subject_path, 'stays.csv'), index_col=None)
    stays.INTIME = pd.to_datetime(stays.INTIME)
    stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
    stays.DOB = pd.to_datetime(stays.DOB)
    stays.DOD = pd.to_datetime(stays.DOD)
    stays.DEATHTIME = pd.to_datetime(stays.DEATHTIME)
    stays.sort_values(by=['INTIME', 'OUTTIME'], inplace=True)
    return stays


def read_diagnoses(subject_path):
    return pd.read_csv(os.path.join(subject_path, 'diagnoses.csv'), index_col=None)


def read_events(subject_path, remove_null=True):
    events = pd.read_csv(os.path.join(subject_path, 'events.csv'), index_col=None)
    if remove_null:
        events = events[events.VALUE.notnull()]
    events.CHARTTIME = pd.to_datetime(events.CHARTTIME)
    events.HADM_ID = events.HADM_ID.fillna(value=-1).astype(int)
    events.ICUSTAY_ID = events.ICUSTAY_ID.fillna(value=-1).astype(int)
    events.VALUEUOM = events.VALUEUOM.fillna('').astype(str)
    # events.sort_values(by=['CHARTTIME', 'ITEMID', 'ICUSTAY_ID'], inplace=True)
    return events

In [143]:
diagnosis_labels = ['4019', '4280', '41401', '42731', '25000', '5849', '2724', '51881', '53081', '5990', '2720',
                    '2859', '2449', '486', '2762', '2851', '496', 'V5861', '99592', '311', '0389', '5859', '5070',
                    '40390', '3051', '412', 'V4581', '2761', '41071', '2875', '4240', 'V1582', 'V4582', 'V5867',
                    '4241', '40391', '78552', '5119', '42789', '32723', '49390', '9971', '2767', '2760', '2749',
                    '4168', '5180', '45829', '4589', '73300', '5845', '78039', '5856', '4271', '4254', '4111',
                    'V1251', '30000', '3572', '60000', '27800', '41400', '2768', '4439', '27651', 'V4501', '27652',
                    '99811', '431', '28521', '2930', '7907', 'E8798', '5789', '79902', 'V4986', 'V103', '42832',
                    'E8788', '00845', '5715', '99591', '07054', '42833', '4275', '49121', 'V1046', '2948', '70703',
                    '2809', '5712', '27801', '42732', '99812', '4139', '3004', '2639', '42822', '25060', 'V1254',
                    '42823', '28529', 'E8782', '30500', '78791', '78551', 'E8889', '78820', '34590', '2800', '99859',
                    'V667', 'E8497', '79092', '5723', '3485', '5601', '25040', '570', '71590', '2869', '2763', '5770',
                    'V5865', '99662', '28860', '36201', '56210']

def extract_diagnosis_labels(diagnoses):
    global diagnosis_labels
    diagnoses['VALUE'] = 1
    labels = diagnoses[['ICUSTAY_ID', 'ICD9_CODE', 'VALUE']].drop_duplicates()\
                      .pivot(index='ICUSTAY_ID', columns='ICD9_CODE', values='VALUE').fillna(0).astype(int)
    for l in diagnosis_labels:
        if l not in labels:
            labels[l] = 0
    labels = labels[diagnosis_labels]
    return labels.rename(dict(zip(diagnosis_labels, ['Diagnosis ' + d for d in diagnosis_labels])), axis=1)

In [144]:
def assemble_episodic_data(stays, diagnoses):
    data = {'Icustay': stays.ICUSTAY_ID, 'Age': stays.age, 'Length of Stay': stays.LOS,
            'Mortality': stays.MORTALITY}
    data.update(transform_gender(stays.GENDER))
    data.update(transform_ethnicity(stays.ETHNICITY))
    data['Height'] = np.nan
    data['Weight'] = np.nan
    data = pd.DataFrame(data).set_index('Icustay')
    data = data[['Ethnicity', 'Gender', 'Age', 'Height', 'Weight', 'Length of Stay', 'Mortality']]
    return data.merge(extract_diagnosis_labels(diagnoses), left_index=True, right_index=True)

In [145]:
g_map = {'F': 1, 'M': 2, 'OTHER': 3, '': 0}
def transform_gender(gender_series):
    return {'Gender': gender_series.fillna('').apply(lambda s: g_map[s] if s in g_map else g_map['OTHER'])}
e_map = {'ASIAN': 1,
         'BLACK': 2,
         'CARIBBEAN ISLAND': 2,
         'HISPANIC': 3,
         'SOUTH AMERICAN': 3,
         'WHITE': 4,
         'MIDDLE EASTERN': 4,
         'PORTUGUESE': 4,
         'AMERICAN INDIAN': 0,
         'NATIVE HAWAIIAN': 0,
         'UNABLE TO OBTAIN': 0,
         'PATIENT DECLINED TO ANSWER': 0,
         'UNKNOWN': 0,
         'OTHER': 0,
         '': 0}
def transform_ethnicity(ethnicity_series):

    def aggregate_ethnicity(ethnicity_str):
        return ethnicity_str.replace(' OR ', '/').split(' - ')[0].split('/')[0]

    ethnicity_series = ethnicity_series.apply(aggregate_ethnicity)
    return {'Ethnicity': ethnicity_series.fillna('').apply(lambda s: e_map[s] if s in e_map else e_map['OTHER'])}


### 只保留events.csv的指定的17个变量的记录，过滤掉其他的events.csv的记录

In [146]:
# 过滤示例：
""" 
a = events.iloc[:10]
print('original a:')
print(a)
b = itemid_map_variable.iloc[:5]
print('b:')
print(b)
a = a.merge(b, left_on='ITEMID', right_index=True)
print('after merged :')
print(a) 
"""

" \na = events.iloc[:10]\nprint('original a:')\nprint(a)\nb = itemid_map_variable.iloc[:5]\nprint('b:')\nprint(b)\na = a.merge(b, left_on='ITEMID', right_index=True)\nprint('after merged :')\nprint(a) \n"

## 需要对events.csv的一些离散数据的value进行处理，比如一些字符串转换为0/1（normal/abnormal）

In [147]:
# CRR: strings with brisk, <3 normal, delayed, or >3 abnormal
'''
毛细血管充盈率： <3为正常, 设置为1
'''
def clean_crr(df):
    v = Series(np.zeros(df.shape[0]), index=df.index)
    v[:] = np.nan

    # when df.VALUE is empty, dtype can be float and comparision with string
    # raises an exception, to fix this we change dtype to str
    df_value_str = df.VALUE.astype(str)

    v.loc[(df_value_str == 'Normal <3 secs') | (df_value_str == 'Brisk')] = 0
    v.loc[(df_value_str == 'Abnormal >3 secs') | (df_value_str == 'Delayed')] = 1
    return v

'''
舒张压：
成人正常的舒张压为60~90mmHg(12kpa) ，血压的单位为千帕，1千帕=7.6mmHg

不知道这里为什么要用到正则匹配，舒张压在mimic里面的难道不是一个数值吗？
回答这个问题：
舒张压分为两部分： systolic（收缩） 和 diastolic（舒张）；mimic是分开存储的，value是数字，
然后，正则匹配接收的类型是：“ 数字 / 数字”

systolic blood pressure，SBP，收缩压；
diastolic blood pressure，DBP，舒张压。
'''   
# SBP: some are strings of type SBP/DBP
def clean_sbp(df):
    v = df.VALUE.astype(str).copy()
    # 找出存储格式是 SBP/DBP的字符串，分割出SBP
    idx = v.apply(lambda s: '/' in s)
    v.loc[idx] = v[idx].apply(lambda s: re.match('^(\d+)/(\d+)$', s).group(1))
    return v.astype(float)

def clean_dbp(df):
    v = df.VALUE.astype(str).copy()
    idx = v.apply(lambda s: '/' in s)
    # # 找出存储格式是 SBP/DBP的字符串，分割出DBP
    v.loc[idx] = v[idx].apply(lambda s: re.match('^(\d+)/(\d+)$', s).group(2))
    return v.astype(float)


# FIO2: many 0s, some 0<x<0.2 or 1<x<20
def clean_fio2(df):
    v = df.VALUE.astype(float).copy()

    ''' The line below is the correct way of doing the cleaning, since we will not compare 'str' to 'float'.
    If we use that line it will create mismatches from the data of the paper in ~50 ICU stays.
    The next releases of the benchmark should use this line.
    '''
    # idx = df.VALUEUOM.fillna('').apply(lambda s: 'torr' not in s.lower()) & (v>1.0)

    ''' The line below was used to create the benchmark dataset that the paper used. Note this line will not work
    in python 3, since it may try to compare 'str' to 'float'.
    '''
    # idx = df.VALUEUOM.fillna('').apply(lambda s: 'torr' not in s.lower()) & (df.VALUE > 1.0)

    ''' The two following lines implement the code that was used to create the benchmark dataset that the paper used.
    This works with both python 2 and python 3.
    '''
    is_str = np.array(map(lambda x: type(x) == str, list(df.VALUE)), dtype=np.bool)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'torr' not in s.lower()) & (is_str | (~is_str & (v > 1.0)))

    v.loc[idx] = v[idx] / 100.
    return v


# GLUCOSE, PH: sometimes have ERROR as value
def clean_lab(df):
    v = df.VALUE.copy()
    idx = v.apply(lambda s: type(s) is str and not re.match('^(\d+(\.\d*)?|\.\d+)$', s))
    v.loc[idx] = np.nan
    return v.astype(float)


# O2SAT: small number of 0<x<=1 that should be mapped to 0-100 scale
def clean_o2sat(df):
    # change "ERROR" to NaN
    v = df.VALUE.copy()
    idx = v.apply(lambda s: type(s) is str and not re.match('^(\d+(\.\d*)?|\.\d+)$', s))
    v.loc[idx] = np.nan

    v = v.astype(float)
    idx = (v <= 1)
    v.loc[idx] = v[idx] * 100.
    return v


# Temperature: map Farenheit to Celsius, some ambiguous 50<x<80
def clean_temperature(df):
    v = df.VALUE.astype(float).copy()
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'F' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'F' in s.lower()) | (v >= 79)
    v.loc[idx] = (v[idx] - 32) * 5. / 9
    return v


# Weight: some really light/heavy adults: <50 lb, >450 lb, ambiguous oz/lb
# Children are tough for height, weight
def clean_weight(df):
    v = df.VALUE.astype(float).copy()
    # ounces
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'oz' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'oz' in s.lower())
    v.loc[idx] = v[idx] / 16.
    # pounds
    idx = idx | df.VALUEUOM.fillna('').apply(lambda s: 'lb' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'lb' in s.lower())
    v.loc[idx] = v[idx] * 0.453592
    return v


# Height: some really short/tall adults: <2 ft, >7 ft)
# Children are tough for height, weight
def clean_height(df):
    v = df.VALUE.astype(float).copy()
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'in' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'in' in s.lower())
    v.loc[idx] = np.round(v[idx] * 2.54)
    return v

# ETCO2: haven't found yet
# Urine output: ambiguous units (raw ccs, ccs/kg/hr, 24-hr, etc.)
# Tidal volume: tried to substitute for ETCO2 but units are ambiguous
# Glascow coma scale eye opening
# Glascow coma scale motor response
# Glascow coma scale total
# Glascow coma scale verbal response
# Heart Rate
# Respiratory rate
# Mean blood pressure


In [148]:
clean_fns = {
    'Capillary refill rate': clean_crr,    # 毛细血管充盈率, 需要处理为0、1，分别代表normal 、 abnormal
    'Diastolic blood pressure': clean_dbp, # 舒张压； 处理SBP/DBP格式的数据，分离出SBP
    'Systolic blood pressure': clean_sbp,  # 收缩压； 处理SBP/DBP格式的数据，分离出DBP
    'Fraction inspired oxygen': clean_fio2,
    'Oxygen saturation': clean_o2sat,
    'Glucose': clean_lab,
    'pH': clean_lab,
    'Temperature': clean_temperature,
    'Weight': clean_weight,
    'Height': clean_height
}


def clean_events(events):
    for var_name, clean_fn in clean_fns.items():
        idx = (events.VARIABLE == var_name)
        try:
            events.loc[idx, 'VALUE'] = clean_fn(events[idx])
        except Exception as e:
            import traceback
            print("Exception in clean_events:", clean_fn.__name__, e)
            print(traceback.format_exc())
            print("number of rows:", np.sum(idx))
            print("values:", events[idx])
            exit()
    return events.loc[events.VALUE.notnull()]

In [149]:
def convert_events_to_timeseries(events, variable_column='VARIABLE', variables=[]):
    # 先把metadata按照时间递增的顺序排列
    metadata = events[['CHARTTIME', 'ICUSTAY_ID']].sort_values(by=['CHARTTIME', 'ICUSTAY_ID'])\
                    .drop_duplicates(keep='first').set_index('CHARTTIME')
    timeseries = events[['CHARTTIME', variable_column, 'VALUE']]\
                    .sort_values(by=['CHARTTIME', variable_column, 'VALUE'], axis=0)\
                    .drop_duplicates(subset=['CHARTTIME', variable_column], keep='last')
    timeseries = timeseries.pivot(index='CHARTTIME', columns=variable_column, values='VALUE')\
                    .merge(metadata, left_index=True, right_index=True)\
                    .sort_index(axis=0).reset_index()
    for v in variables:
        if v not in timeseries:
            timeseries[v] = np.nan
    return timeseries

In [150]:
def get_events_for_stay(timeseries, icustayid, intime=None, outtime=None):
    idx = (timeseries.ICUSTAY_ID == icustayid)
    # 这种情况避免了ICUSTAY_ID为空的情况
    if intime is not None and outtime is not None:
        idx = idx | ((timeseries.CHARTTIME >= intime) & (timeseries.CHARTTIME <= outtime))
    timeseries = timeseries[idx]
    del timeseries['ICUSTAY_ID']
    return timeseries

In [151]:
def get_first_valid_from_timeseries(timeseries, variable):
    if variable in timeseries:
        idx = timeseries[variable].notnull()
        # 判断'Height'是否有非空的取值
        if idx.any():
            loc = np.where(idx)[0][0]
            return timeseries[variable].iloc[loc]
    return np.nan

In [152]:
def add_hours_elpased_to_events(events, dt, remove_charttime=True):
    events = events.copy()
    # 计算测量某项的操作是入住ICU多少小时后进行的
    events['HOURS'] = (events.CHARTTIME - dt).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60
    if remove_charttime:
        del events['CHARTTIME']
    return events

In [154]:
for subject_dir in tqdm(os.listdir(subjects_root_path), desc='Iterating over subjects'):
    #subject_dir = str(3)
    dn = os.path.join(subjects_root_path, subject_dir)
    try:
        subject_id = int(subject_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        sys.stderr.write(' {} doesnot exists\n'.format(subject_id))

    try:
        # reading tables of this subject
        stays = read_stays(os.path.join(subjects_root_path, subject_dir))
        #print(stays)
        diagnoses = read_diagnoses(
            os.path.join(subjects_root_path, subject_dir))
        #print('diagnose:')
        #print(diagnoses)
        events = read_events(os.path.join(subjects_root_path, subject_dir))
    except:
        sys.stderr.write(
            'Error reading from disk for subject: {}\n'.format(subject_id))
    # episodic_data是ICU信息和诊断信息的结合
    episodic_data = assemble_episodic_data(stays, diagnoses)
    # 只保留events.csv的指定的17个变量的记录，过滤掉其他的events.csv的记录
    '''
    itemid_map_variable的内容是17个变量的itemid、benchmark_name、mimic_label;
    itemid_map_variable的index是ITEMID
    '''
    events = events.merge(itemid_map_variable,
                          left_on='ITEMID', right_index=True)
    # 处理一下17个变量里面的部分变量的取值（清洗数据的过程）
    events = clean_events(events)
    
    # 把events的内容按照时间递增的顺序排序，每一行包含17个列（17个变量）
    # variables是17个临床变量所使用的的名字
    timeseries = convert_events_to_timeseries(
        events, variable_column='VARIABLE', variables=variables)
    # print(timeseries)

    # extracting separate episodes
    # extracting separate episodes
    # 遍历患者的每一次住院记录（很多病人只有1次住院记录）
    for i in range(stays.shape[0]):
        stay_id = stays.ICUSTAY_ID.iloc[i]
        intime = stays.INTIME.iloc[i]
        outtime = stays.OUTTIME.iloc[i]
        # 得到某病人某次入住ICU的17个变量的测量记录
        # 即episode是某个病人某次入住ICU的17个变量的所有测量记录
        episode = get_events_for_stay(timeseries, stay_id, intime, outtime)
        if episode.shape[0] == 0:
            # no data for this episode
            continue
        # 计算测量某项的操作是入住ICU多少小时后进行的，并将递增的hour作为index
        episode = add_hours_elpased_to_events(
            episode, intime).set_index('HOURS').sort_index(axis=0)
        if stay_id in episodic_data.index:
            episodic_data.loc[stay_id, 'Weight'] = get_first_valid_from_timeseries(
                episode, 'Weight')
            episodic_data.loc[stay_id, 'Height'] = get_first_valid_from_timeseries(
                episode, 'Height')
        # episodic_data是ICU信息和诊断信息的结合
        episodic_data.loc[episodic_data.index == stay_id].to_csv(os.path.join(subjects_root_path, subject_dir,
                                                                              'episode{}.csv'.format(i+1)),
                                                                 index_label='Icustay')
        columns = list(episode.columns)
        columns_sorted = sorted(columns, key=(
            lambda x: "" if x == "Hours" else x))
        #print(episode)
        episode = episode[columns_sorted]
        episode.to_csv(os.path.join(subjects_root_path, subject_dir, 'episode{}_timeseries.csv'.format(i+1)),
                       index_label='Hours')
        

Iterating over subjects:   0%|          | 24/33798 [00:04<1:40:05,  5.62it/s]


KeyboardInterrupt: 